# Cleaning Tags

In [1]:
# Importing libraries
from keras.utils.np_utils import to_categorical

import pandas as pd
import numpy as np

import re

import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from itertools import combinations
from sklearn.cluster import KMeans

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import tensorflow as tf
import tensorflow.compat.v1 as tf
import tensorflow_hub as hub
import numpy as np

import matplotlib.pyplot as plt
import nltk

nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('stopwords')

nltk.download('punkt')

import spacy 
from spacy import displacy

import random

from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Embedding,Dense

import os
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorboard.plugins import projector

from gensim.scripts.glove2word2vec import glove2word2vec

# import en_core_web_lg
# nlp = en_core_web_lg.load()

from nltk.tokenize import word_tokenize

import gensim
from scipy import spatial

import csv
import tensorflow as tf

import multiprocessing
import math

import time
from gensim.models import KeyedVectors
from gensim.models.fasttext import FastText
import pandas as pd
import gensim
gensim.__version__

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


'3.6.0'

In [5]:
#importing the tags dataset
data = pd.read_csv("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/ml-25m/movie_all_info.csv")
tags = pd.read_csv("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/ml-25m/tags.csv")
#Importing dataframe which has tags for movies which were originally missing tags in the MovieLens dataset (created by Anhui)
notag_movies = pd.read_csv("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/ml-25m/notag_movie.csv")

In [6]:
tags.head(2)

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256


# MOVIES

#### Combining Tags

In [7]:
#combining all tags for one movie into a single entry
tags['tag'] = tags['tag'].astype(str)
tags['combined_tags'] = tags.groupby('movieId')['tag'].transform(lambda x: '|'.join(x))
movie_tags = tags[['movieId', 'combined_tags']]
movie_tags.head()

#merging the tags with the movie_all info datasets
data2 = data.merge(movie_tags, on='movieId', how='left').drop_duplicates().reset_index().drop(['index'], axis = 1)

#extracting genres and tags from the main dataframe
movtag = data2[['movieId', 'genres', 'combined_tags']]

In [8]:
movtag.isnull().sum()

movieId              0
genres              11
combined_tags    17172
dtype: int64

We dont have tags for almost 17k titles in the dataset. Is there any way to fill this gap? We can take the genome tags for the movies.

In [10]:
#adding tags which were missing
movtag['combined_tags'] = movtag['movieId'].map(notag_movies.set_index('movieId')['tag']).fillna(movtag['combined_tags'])
movtag.isnull().sum()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


movieId            0
genres            11
combined_tags    454
dtype: int64

In [11]:
movtag.head()

,movieId,genres,combined_tags
0,1,Adventure|Animation|Children|Comedy|Fantasy,Owned|imdb top 250|Pixar|Pixar|time travel|chi...
1,2,Adventure|Children|Fantasy,Robin Williams|time travel|fantasy|based on ch...
2,3,Comedy|Romance,funny|best friend|duringcreditsstinger|fishing...
3,4,Comedy|Drama|Romance,based on novel or book|chick flick|divorce|int...
4,5,Comedy,aging|baby|confidence|contraception|daughter|g...


### FEATURE ENGINEERING WITH GENRES AND TAGS

We will split the genres and tags into words and try to tokenise them to see if we can extract any meanings from them

In [12]:
#Separating out tags and genres based on "|"
movtag1 = movtag.copy()
movtag1['genres'] = movtag1['genres'].apply(lambda x: re.split("\|", str(x)))
movtag1['combined_tags'] = movtag1['combined_tags'].apply(lambda x: re.split("\|",str(x)))

#converting all text into lower case
# movtag1['genres'] = movtag1['genres'].apply(lambda x: [word.lower() for word in x])
# movtag1['combined_tags'] = movtag1['combined_tags'].apply(lambda x: [word.lower() for word in x])

#removing duplicates from genres and combined_tags
movtag1['genres_nodup'] = movtag1['genres'].apply(lambda x: [i for n, i in enumerate(x) if i not in x[:n]])
movtag1['combined_tags_nodup'] = movtag1['combined_tags'].apply(lambda x: [i for n, i in enumerate(x) if i not in x[:n]])

#removing stop words and punctuations and lemmatising words
stop = set(nltk.corpus.stopwords.words('english'))

movtag1['combined_tags_nodup'] = movtag1['combined_tags_nodup'].apply(lambda x: ' '.join(x))
movtag1['combined_tags_nodup'] = movtag1['combined_tags_nodup'].apply(lambda x: word_tokenize(x))
movtag1['combined_tags_nodup'] = movtag1['combined_tags_nodup'].apply(lambda x: [word for word in x if word not in stop])
movtag1['combined_tags_nodup'] = movtag1['combined_tags_nodup'].apply(lambda x: [word for word in x if len(word) > 1])
movtag1['combined_tags_nodup'] = movtag1['combined_tags_nodup'].apply(lambda x: [word for word in x if not word.isnumeric()])

movtag2 = movtag1.copy()

In [13]:
movtag2.head()

,movieId,genres,combined_tags,genres_nodup,combined_tags_nodup
0,1,"[Adventure, Animation, Children, Comedy, Fantasy]","[Owned, imdb top 250, Pixar, Pixar, time trave...","[Adventure, Animation, Children, Comedy, Fantasy]","[Owned, imdb, top, Pixar, time, travel, childr..."
1,2,"[Adventure, Children, Fantasy]","[Robin Williams, time travel, fantasy, based o...","[Adventure, Children, Fantasy]","[Robin, Williams, time, travel, fantasy, based..."
2,3,"[Comedy, Romance]","[funny, best friend, duringcreditsstinger, fis...","[Comedy, Romance]","[funny, best, friend, duringcreditsstinger, fi..."
3,4,"[Comedy, Drama, Romance]","[based on novel or book, chick flick, divorce,...","[Comedy, Drama, Romance]","[based, novel, book, chick, flick, divorce, in..."
4,5,[Comedy],"[aging, baby, confidence, contraception, daugh...",[Comedy],"[aging, baby, confidence, contraception, daugh..."


In [18]:
movtag2.to_pickle("/content/drive/MyDrive/MScA - UChicago/Assignments/Quarter 3 - Spring 2022/Capstone Project/Final datasets/1_cleaned_tags.pkl")